In [1]:
# https://github.com/spmallick/learnopencv/tree/master/Keras-ImageNet-Models
# https://www.learnopencv.com/keras-tutorial-using-pre-trained-imagenet-models/

# https://github.com/spmallick/learnopencv/tree/master/Keras-Transfer-Learning
# https://www.learnopencv.com/keras-tutorial-transfer-learning-using-pre-trained-models/

# https://github.com/spmallick/learnopencv/tree/master/Keras-Fine-Tuning
# https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img

Using TensorFlow backend.


In [3]:
train_dir = './clean_data/train'
validation_dir = './clean_data/valid'
image_size = 224

In [4]:
from keras.applications import VGG16

#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Freeze all the layers
for layer in vgg_conv.layers[:]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)


from keras import models
from keras import layers
from keras import optimizers

# Create the model
model = models.Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

<keras.engine.input_layer.InputLayer object at 0x7f12ac4cf278> False
<keras.layers.convolutional.Conv2D object at 0x7f12ac4cf7f0> False
<keras.layers.convolutional.Conv2D object at 0x7f12ac4cf668> False
<keras.layers.pooling.MaxPooling2D object at 0x7f12ac4f0860> False
<keras.layers.convolutional.Conv2D object at 0x7f12ac4f06a0> False
<keras.layers.convolutional.Conv2D object at 0x7f12ac4ac160> False
<keras.layers.pooling.MaxPooling2D object at 0x7f12ac448438> False
<keras.layers.convolutional.Conv2D object at 0x7f12ac4486d8> False
<keras.layers.convolutional.Conv2D object at 0x7f12ac469d68> False
<keras.layers.convolutional.Conv2D object at 0x7f12ac480fd0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f12ac4365c0> False
<keras.layers.convolutional.Conv2D object at 0x7f12ac436438> False
<keras.layers.convolutional.Conv2D object at 0x7f12ac3d4d68> False
<keras.layers.convolutional.Conv2D object at 0x7f12ac3ecfd0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f12ac3a15c0>

In [8]:
# No Data augmentation 
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Change the batchsize according to your system RAM
train_batchsize = 20
val_batchsize = 10

# Data Generator for Training data
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')

# Data Generator for Validation data
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 900 images belonging to 3 classes.
Found 180 images belonging to 3 classes.


In [9]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [10]:
# Train the Model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=20,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor of shape [25088,1024] and type float
	 [[node training_1/RMSprop/zeros (defined at /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:702)  = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [25088,1024] values: [0 0 0...]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'training_1/RMSprop/zeros', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch

In [ ]:
# Save the Model
model.save('all_freezed.h5')

# Plot the accuracy and loss curves
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Up Next:

#### https://github.com/fchollet/deep-learning-with-python-notebooks